In [1]:
from pysat.formula import CNF
import sys
import os
import math

In [2]:
import pysat
pysat.__version__

'0.1.7.dev12'

In [3]:
test_sats_path = 'test sudokus'

In [4]:
def read_DIMACS(path):
    return CNF(from_file=path).clauses
    
    
rules = read_DIMACS(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'))

In [5]:
input_mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9, 'A': 10, 'B': 11, "C": 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16}

inverse_input_mapping = {v:k for k, v in input_mapping.items()}

In [6]:
def line2DIMACS(sudoku):
    input_len = len(sudoku)
    table_size = int(math.pow(input_len, 0.5))
    clause_list = []
    for row in range(table_size):
        for column in range(table_size):
            current_line_index = column + table_size * row
            cell_value = sudoku[current_line_index]
            if cell_value != '.':
                clause_list.append(f"{inverse_input_mapping[row+1]}{inverse_input_mapping[column+1]}{cell_value} 0")
    return clause_list, table_size

def line2CNF(line):
    cnf_list, table_size = line2DIMACS(line)
    return CNF(from_string='\n'.join(cnf_list)).clauses, cnf_list, table_size

In [7]:
# with open(os.path.join(test_sats_path, '4x4.txt')) as f:
with open(os.path.join(test_sats_path, '1000 sudokus.txt')) as f:
    sudoku_list = f.readlines()

In [8]:
sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku_list[1])

In [9]:
from dpll import DPLL

In [10]:
clauses = rules + sudoku_setup_CNF
solver = DPLL(clauses, variable_selection_method='random', verbose=0)

In [11]:
# solver.variables_set

In [12]:
# solver.clauses

In [13]:
%%time

solution = solver.backtrack(solver.clauses, partial_assignment=[], split_literal=tuple())

CPU times: user 22.1 s, sys: 158 ms, total: 22.3 s
Wall time: 22.5 s


In [14]:
solution[0]

True

In [15]:
def get_digit(number, n):
    return int(number // 10**n % 10)

import numpy as np
matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for clause in sudoku_setup_CNF:
    for item in clause:
        matrix[get_digit(item, 2)-1, get_digit(item, 1)-1] = get_digit(item, 0)

In [16]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
matprint(matrix)

7  0  8  9  0  0  0  0  0  
0  0  6  0  0  8  0  4  0  
0  0  0  2  5  0  0  3  0  
0  0  0  0  0  0  1  0  0  
6  0  0  0  1  0  0  2  0  
0  7  0  0  0  5  0  6  0  
0  0  0  0  7  0  0  0  0  
9  2  0  0  0  3  0  0  0  
1  0  0  0  8  0  6  0  0  


In [17]:
found_assignments = sorted(solution[1], key=lambda x: x[0])

matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for assignment in found_assignments:
    if assignment[1]:
        item = assignment[0]
        matrix[int(item[0])-1, int(item[1])-1] = int(item[2])
        
matprint(matrix)

7  3  8  9  4  1  2  5  6  
2  5  6  7  3  8  9  4  1  
4  1  9  2  5  6  7  3  8  
5  8  2  6  9  4  1  7  3  
6  9  4  3  1  7  8  2  5  
3  7  1  8  2  5  4  6  9  
8  6  5  4  7  9  3  1  2  
9  2  7  1  6  3  5  8  4  
1  4  3  5  8  2  6  9  7  


# Experiment

In [35]:
import random
random.seed(42)
import pandas as pd

In [56]:
import time
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [60]:
def run_experiment(rules_path, setups_path, sudoku_num=50):
    rules = read_DIMACS(rules_path)
    
    with open(setups_path) as f:
        sudoku_list = f.readlines()
    sudoku_sample_indices = random.sample(range(0, len(sudoku_list)), sudoku_num)
    
    results_df = pd.DataFrame(columns=['rules_path', 'setups_path', 'setup_index', 'sat', 'solution', 'time'])

    for sudoku_index in sudoku_sample_indices:
        sudoku = sudoku_list[sudoku_index]
        
        sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku)
        
        clauses = rules + sudoku_setup_CNF
        solver = DPLL(clauses, variable_selection_method='random', verbose=0)
        
        start = time.time()
        solution = solver.backtrack(solver.clauses, partial_assignment=[], split_literal=tuple())     
        solution_time = time.time() - start
        
        results_df.loc[len(results_df), ['rules_path', 'setups_path', 'setup_index', 'sat', 'solution', 'time']] = rules_path, setups_path, sudoku_index, *solution, solution_time
        display(results_df.tail(1))
        
    return results_df

In [62]:
results_df = run_experiment(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'),
                            os.path.join(test_sats_path, '1000 sudokus.txt'), # 1000 sudokus, 4x4
                            sudoku_num=10,
              )

,rules_path,setups_path,setup_index,sat,solution,time
0,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,634,True,"[(124, True), (121, False), (122, False), (123...",27.263326


,rules_path,setups_path,setup_index,sat,solution,time
1,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,83,True,"[(118, True), (111, False), (112, False), (113...",34.89422


,rules_path,setups_path,setup_index,sat,solution,time
2,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,429,True,"[(129, True), (121, False), (122, False), (123...",27.252194


,rules_path,setups_path,setup_index,sat,solution,time
3,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,673,True,"[(115, True), (111, False), (112, False), (113...",28.723595


,rules_path,setups_path,setup_index,sat,solution,time
4,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,597,True,"[(119, True), (111, False), (112, False), (113...",32.438397


,rules_path,setups_path,setup_index,sat,solution,time
5,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,578,True,"[(132, True), (131, False), (133, False), (134...",31.983585


,rules_path,setups_path,setup_index,sat,solution,time
6,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,535,True,"[(115, True), (111, False), (112, False), (113...",36.344936


,rules_path,setups_path,setup_index,sat,solution,time
7,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,323,True,"[(216, True), (211, False), (212, False), (213...",33.584052


,rules_path,setups_path,setup_index,sat,solution,time
8,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,957,True,"[(134, True), (131, False), (132, False), (133...",32.794243


,rules_path,setups_path,setup_index,sat,solution,time
9,test sudokus/sudoku-rules-9x9.txt,test sudokus/1000 sudokus.txt,267,True,"[(143, True), (141, False), (142, False), (144...",31.606429


In [63]:
results_df.to_csv("10_launches_size_9.csv", index=False)

In [65]:
class Counter(object) :
    def __init__(self, fun) :
        self._fun = fun
        self.counter=0
    def __call__(self,*args, **kwargs) :
        self.counter += 1
        return self._fun(*args, **kwargs)
    
def recur(n) :
    print('recur', n)
    if n>0 :
        return recur(n-1)
    return 0

recur = Counter(recur)

recur(5)

recur 5
recur 4
recur 3
recur 2
recur 1
recur 0


0

In [66]:
recur.counter

6

In [67]:
@Counter
def recur2(n):
    print('recur', n)
    if n>0 :
        return recur(n-1)
    return 0

In [68]:
recur2(5)

recur 5
recur 4
recur 3
recur 2
recur 1
recur 0


0

In [ ]:
recur2.c